
##Generate a labelled 1000-row text dataset using OpenAI’s ChatGPT. This dataset should contain positive and negative reviews of something (e.g, you could ask ChatGPT to generate positive/ negative movie reviews, or positive/ negative smartphone reviews, etc) You might have to run your prompt multiple times to compile a 1000-row dataset as ChatGPT won’t give you 1000 reviews in one go. Name this dataset Reviews.csv

###**Importing Library**### 

In [ ]:
!pip3 install umap-learn
import re, nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import plotly.graph_objs as go
import plotly.figure_factory as ff
import umap
from matplotlib import pyplot as plt
from umap import UMAP
!pip install umap-learn[plot]
!pip install holoviews
!pip install -U ipykernel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2023-04-23 21:23:45.719484: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 21:23:50.941708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##**Importing Data**

In [ ]:
import pandas
df = pandas.read_excel('/content/Reviews.xlsx')


In [ ]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Song,Genre,Review,Rating (Positive/Negative)
1,"""Shape of You""",Pop,"""This song has a great beat and catchy lyrics....",Positive
2,"""Rolling in the Deep""",Pop,"""Adele's powerful vocals and emotional lyrics ...",Positive
3,"""Bad Guy""",Alternative/Indie,"""Billie Eilish's unique style and haunting voc...",Positive
4,"""Hello""",Pop,"""Adele's comeback hit is everything we wanted ...",Positive
...,...,...,...,...
1024,Dil Diyan Gallan,Hindi,"Such a romantic and soulful song, one of my fa...",Positive
1025,Jag Ghoomeya,Hindi,"Not a fan of this song, it's too slow and bori...",Negative
1026,Aashiqui 2 Mashup,Hindi,"Love the remix of this song, it's even better ...",Positive
1027,Chaiyya Chaiyya Remix,Hindi,"Love the modern twist on this classic song, it...",Positive


In [ ]:
df.columns = ['Song', 'Genre', 'Reviews','Ratings']

###**Cleaning Data**

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df = df.drop(labels=0, axis=0)

In [ ]:
df.shape

(1028, 4)

In [ ]:
df2 = df.drop_duplicates(subset=["Song"])


In [ ]:
df2.shape

(916, 4)

In [ ]:
data = df2.dropna()


In [ ]:
data


,Song,Genre,Reviews,Ratings
1,"""Shape of You""",Pop,"""This song has a great beat and catchy lyrics....",Positive
2,"""Rolling in the Deep""",Pop,"""Adele's powerful vocals and emotional lyrics ...",Positive
3,"""Bad Guy""",Alternative/Indie,"""Billie Eilish's unique style and haunting voc...",Positive
4,"""Hello""",Pop,"""Adele's comeback hit is everything we wanted ...",Positive
5,"""Old Town Road""",Country,"""This song is so fun and catchy! I love the bl...",Positive
...,...,...,...,...
1024,Dil Diyan Gallan,Hindi,"Such a romantic and soulful song, one of my fa...",Positive
1025,Jag Ghoomeya,Hindi,"Not a fan of this song, it's too slow and bori...",Negative
1026,Aashiqui 2 Mashup,Hindi,"Love the remix of this song, it's even better ...",Positive
1027,Chaiyya Chaiyya Remix,Hindi,"Love the modern twist on this classic song, it...",Positive


In [ ]:
data.shape

(908, 4)

###**Feature Engineering**

In [ ]:
data['Ratings'] = data['Ratings'].map({'Positive':1, 'Negative':0})

###**TFIDF AND UMAP Dimentionality Reduction** 

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=2)
tfidf_matrix = tfidf.fit_transform(data['Reviews'])
reducer = umap.UMAP(n_neighbors=5, min_dist=0.3)
embedding = reducer.fit_transform(tfidf_matrix)

In [ ]:
Y = data['Ratings']

In [ ]:
ratings = list(data['Ratings'])
reviews = list(data['Reviews'])

**Task 2** - How does tuning TF-IDF hyperparameters ‘ngram_range’ and ‘min_df’ affect the TF-IDF matrix and the subsequent visualization? 

**Ans**. The TF-IDF (Term Frequency-Inverse Document Frequency) matrix is a numerical text data representation that shows the relevance of each term in a document corpus. The TF-IDF matrix and subsequent display can be considerably influenced by hyperparameters such as ngram_range and min_df.

ngram_range: This option specifies how many consecutive words, or n-grams, are treated as a single phrase. The most frequent default setting is (1,1), which indicates that only single words are examined. Increasing ngram_range to (1,2) or (2,2) will contain bi-grams, or pairs of consecutive words, or tri-grams (3,3) for triples of consecutive words, and so on.


Larger n-grams capture more context and meaning in the text, but they also increase the dimensionality of the TF-IDF matrix, making it more difficult to read.When employing bigger n-grams, visualization approaches such as word clouds or heatmaps may become less effective since the resulting matrix contains too many terms to display.

min_df: This option defines the minimal number of documents required for a phrase to appear in the TF-IDF matrix. It is used to exclude terms that are uncommon or unimportant to the analysis.

By increasing min_df, terms that appear in fewer documents are excluded, resulting in a smaller and more focused TF-IDF matrix. This can help to minimize noise and increase analytical accuracy. Setting min_df too high, on the other hand, may remove significant terms that are uncommon but still relevant. Lowering min_df allows for the inclusion of additional words, but it might also cause problems.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
data = [go.Scatter(x=embedding[:, 0], y=embedding[:, 1],mode='markers',
                    marker = dict(color=Y, colorscale='Rainbow', opacity=0.5),
                               
                                hoverinfo='text')]

layout = go.Layout(title = 'UMAP Dimensionality Reduction', width = 800, height = 800,
                    xaxis = dict(title='First Dimension'),
                    yaxis = dict(title='Second Dimension'))
fig = go.Figure(data=data, layout=layout)
fig.show()

**Task 2***

**Ans**The dimension reduction technique UMAP (Uniform Manifold Approximation and Projection) is often used for presenting high-dimensional data in a lower-dimensional domain. Hyperparameters like n_neighbors and min_dist can have a big impact on the final display.

n_neighbors: This parameter specifies the number of nearby points in the lower-dimensional space to which each point in the original high-dimensional space is related.

Increasing n_neighbors can lead to a more faithful representation of the original data in the lower-dimensional space, since each point is influenced by more neighbors. However, increasing n_neighbors can also increase the computational complexity of the algorithm, which can make it slower to compute. It can also lead to overcrowding of the visualization since more points will be clustered together.

min_dist: This option regulates how densely points are packed in lower-dimensional space. It denotes the shortest distance between points in lower-dimensional space.

Reduced min_dist can result in a more compact and dense display with better-defined clusters. Setting min_dist too low, on the other hand, can cause points to overlap, resulting in a distorted or deceptive image. Increasing min_dist can assist to spread out the points and make the visualization more obvious, but it might also eliminate crucial local structure in the data.




To summarize, the n_neighbors and min_dist hyperparameters can have an effect on the quality and interpretability of the final UMAP display. The ideal hyperparameters should be chosen based on the analysis's specific aims and the features of the data being examined. To determine the optimal values for a given dataset, it is often advised to experiment with different choices of n_neighbors and min_dist.

**Task 3** - 

**Ans** - The sub-clusters inside positive and negative reviews provide information about the many subjects or themes prevalent in the reviews. For example, we might find reviews regarding the melody or singing in the positive reviews sub-clusters, while we might find reviews concerning the lyrics or production quality in the bad reviews sub-clusters. These sub-clusters can be utilized to highlight areas where the song can be improved.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(embedding)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(n_clusters=2, random_state=42)

In [ ]:
 from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD

# reduce the dimensionality of the TF-IDF matrix using TruncatedSVD
svd = TruncatedSVD(n_components=50)
X_svd = svd.fit_transform(tfidf_matrix)

# cluster the reviews based on positive and negative sentiments
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X_svd)

# get the labels for each review
labels = kmeans.labels_

pos_indices = [i for i, label in enumerate(labels) if label == 0]
neg_indices = [i for i, label in enumerate(labels) if label == 1]

# reduce the dimensionality of the positive and negative reviews separately
pos_reviews_svd = svd.transform(tfidf_matrix[pos_indices])
neg_reviews_svd = svd.transform(tfidf_matrix[neg_indices])

# cluster the positive reviews into sub-clusters
kmeans_pos = KMeans(n_clusters=2, random_state=42)
kmeans_pos.fit(pos_reviews_svd)

# get the labels for each positive review
pos_labels = kmeans_pos.labels_

# cluster the negative reviews into sub-clusters
kmeans_neg = KMeans(n_clusters=2, random_state=42)
kmeans_neg.fit(neg_reviews_svd)

# get the labels for each negative review
neg_labels = kmeans_neg.labels_


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

